In [14]:
import daglet
from functools import partial
from ipywidgets import Image
from IPython.display import display
from PIL import Image as PILImage
import io
import hcl


def get_image_size(image_data):
    file = io.BytesIO(image_data)
    return PILImage.open(file).size


class Dag(object):
    def __init__(self, vertices=[]):
        self.vertices = vertices

    def extend(self, value):
        vertex = Vertex(value, self.vertices)
        return Dag([vertex])

    def merge(self, *others):
        dags = [self] + others
        vertices_list = [x.vertices for x in dags]
        vertices = reduce(lambda x, y: x + y, vertices_list)
        return Dag(vertices)

    def __or__(self, value):
        return self.extend(value)

    def __xor__(self, other):
        return self.merge(other)

    
class Vertex(object):
    def __init__(self, value, parents=[]):
        self.value = value
        self.parents = parents
        
    def get_parents(self):
        return self.parents
    

print(hash(Vertex(None)))
print(hash(Vertex(None)))

    
def get_vertex_label_map():    
    keys = globals().keys()
    keys = [x for x in keys if not x.startswith('_')]

    vertex_label_map = {}
    for key in keys:
        value = globals()[key]
        if isinstance(value, Dag):
            for vertex in value.vertices:
                vertex_label_map.setdefault(vertex, []).append(key)
        elif isinstance(value, Vertex):
            vertex_label_map.setdefault(value, []).append(key)
    return vertex_label_map


def show_image(image_data, scale=0.7):
    width, height = get_image_size(image_data)
    width *= scale
    height *= scale
    image = Image(value=image_data, width=width, height=height)
    display(image)


def show_all(scale=0.7):
    vertex_label_map = get_vertex_label_map()
    vertices = list(vertex_label_map.keys())
    get_repr = lambda x: x.value
    vertex_refs = {vertex: ', '.join(label) for vertex, label in vertex_label_map.items()}
    get_label = lambda vertex: '{}\n\n{}'.format(get_repr(vertex), vertex_refs.get(vertex, ''))
    get_color = lambda x: '#ffcc00'

    image_data = daglet.render(
        vertices,
        parent_func=Vertex.get_parents,
        vertex_label_func=get_label,
        vertex_color_func=get_color,
    )

    show_image(image_data, scale)


class DurkNode(object):
    def __init__(self, kind, *args):
        self.kind = kind
        self.args = args
        
    def compile(self):
        return '{} {}'.format(self.kind.upper(), ' '.join(self.args))


class durk(object):
    @staticmethod
    def compile(durk_dag):
        vertices = daglet.toposort(durk_dag.vertices, Vertex.get_parents)
        durk_nodes = [x.value for x in vertices]
        return '\n'.join([x.compile() for x in durk_nodes])
    
    @staticmethod
    def from_tag(*args):
        return Dag() | durk._from(*args)


def make_durk_func(kind):
    return lambda *args, **kwargs: DurkNode(kind, *args, **kwargs)


kinds = ['from', 'run', 'workdir', 'copy', 'cmd']
kind_key_map = {'from': '_from'}
for kind in kinds:
    key = kind_key_map.get(kind, kind)
    setattr(durk, key, make_durk_func(kind))
    
    
def inc(x):
    return x + 1

def dec(x):
    return x - 1

def add(x, y):
    return x + y

out = (
    durk.from_tag('frolvlad/alpine-python3')
    | durk.run('pip install flask')
    | durk.workdir('/app')
    | durk.copy('app.py', '.')
    | durk.cmd('flask run')
)

show_all()

print(durk.compile(out))

285681819
285534605


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x08\x1d\x00\x00\x00P\x08\x06\x00\x00\x00_\x96\x9f\xf…

FROM frolvlad/alpine-python3
RUN pip install flask
WORKDIR /app
COPY app.py .
CMD flask run


In [13]:
class TfNode(object):
    def __init__(self, tf_dict, path=''):
        self.tf_dict = tf_dict
        self.path = path
    
    def get_children(self):
        children = []
        if isinstance(self.tf_dict, dict):
            for key, child_dict in self.tf_dict.items():
                child_path = '.'.join([self.path, key]) if self.path else key
                children.append(TfNode(child_dict, child_path))
        return children
    
    def __hash__(self):
        return daglet.utils.get_hash_int([self.tf_dict, self.path])

    def __eq__(self, other):
        return hash(self) == hash(other)
    

with open('main.tf') as f:
    tf_data = hcl.load(f)

def xform(node, parent_values):
    return Vertex(node.path, parent_values)

root = TfNode(tf_data)
vertex_map = daglet.transform_vertices([root], TfNode.get_children, xform)
out = vertex_map[root]
show_all()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\xfe\x00\x00\x05 \x08\x06\x00\x00\x00I\xa1\xff*\x…